# Introduction

This is a set of tutorials over two sessions to give an overview of advanced computing topics. It is intended as an introductory material, to 'whet you appetite', so that you can explore these topics even further as you move along and do great things in research, whereever it may be even beyond CMS.

Given we have only two sessions, they cannot be expected to cover all related topics exhaustively, but the topics I have selected cover a story that will allow you to build heterogeneous computing architectures. Ultimately, as fellow researchers, you would need to do your own investigation and develop upon this material in your own time, but hopefully, this material will act as a seed of initial conditions.

The topics can be vaguely separated in three categories:
- Advanced C++ features: As much as it may sometimes be less practical to do coding with this language in terms of personal time, it is a central feature in any advanced programming interface you will encounter today, whether it may be CUDA, TensorFlow, *hls4ml*, or any other scientific resource (well, you could also learn assembly). We will be exploring concepts that progressively go beyond basic functions or class inheritance.
- Automatic differentiation: This is a concept that is regularly used in machine learning algorithms. Have you ever wondered what happens behind the scenes in TensorFlow? You will take a glimpse at it by the end of these tutorials. Automatic differentiation has also been considered to augment detector response and statistical analysis, see these papers [[1](https://arxiv.org/abs/2003.12483)] and [[2](https://arxiv.org/abs/2204.13621)] for example, so it is not a bad idea to have some practice with it.
- GPU programming: I could have started with abstraction interfaces such as Alpaka and Kokkos (we use Alpaka in CMS) directly. In the next decade, however, thanks to cross-industry efforts like the UXL Foundation, you may even find these interfaces get replaced by improved programming models to handle heterogeneous computing seamlessly. For that reason, it is more important to know the basics and get ready for changes in programming (and understand why they happen); that would also help you learn these abstraction interfaces in the first place anyway. There are many training resources available online, and many of them are even free of charge. I have picked [CUDA](https://developer.nvidia.com/cuda-toolkit) from NVIDIA as a way to introduce you to these concepts, but I could have equally picked [DPC++](https://www.intel.com/content/www/us/en/developer/videos/dpc-part-1-introduction-to-new-programming-model.html#gs.6ks805) from Intel oneAPI, or [ROCm](https://rocm.docs.amd.com/en/latest/) from AMD.


**What do you need for this tutorial?** - Either a connection to `lxplus`/`lxplus-gpu` nodes at CERN, or Python, CUDA, and TensorFlow installed in your computer.

**Where can I find the exercises/solutions?** - If you are actively participating in the tutorial right now, we will be actively communicating them. If you are connecting offline, the link to the material is posted on the Indico agenda item for the corresponding tutorial session.

<div style="text-align: right">U. Sar&#305;ca, March 2024</div>

## Warm-up: Recursive functions

Let's do a warm-up exercise.

Suppose you have 3 poles. On the leftmost pole, you have $n$ number of disks stacked on top of each other in decreasing radius. You need to stack the disks in the same order on the right pole.
- Each move can only happen from the top of the pole.
- During the transfer, a disk with a larger radius can never be stacked on top of one with a smaller radius.

Find the shortest sequence for $n=2$. $3$, and $4$.

### Solution

This is called the *Tower of Hanoi* problem. The basic recursive formulation for $n$ disks is to transfer $(n-1)$ of those at the top to the spare pole, transfer the $n^\text{th}$ disk to the destination, and then transfer the $n-1$ stack to the destination.

In [1]:
def operation(n, src, dest):
  aux = 3 - src - dest
  if n == 1:
    print(f"Move disk 1 from rod {src} to rod {dest}.")
    return
  operation(n - 1, src, aux)
  print(f"Move disk {n} from rod {src} to rod {dest}.")
  operation(n - 1, aux, dest)

print("Operations for n = 2:")
operation(2, 0, 2)

print("Operations for n = 3:")
operation(3, 0, 2)

print("Operations for n = 4:")
operation(4, 0, 2)

Operations for n = 2:
Move disk 1 from rod 0 to rod 1.
Move disk 2 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 2.
Operations for n = 3:
Move disk 1 from rod 0 to rod 2.
Move disk 2 from rod 0 to rod 1.
Move disk 1 from rod 2 to rod 1.
Move disk 3 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 0.
Move disk 2 from rod 1 to rod 2.
Move disk 1 from rod 0 to rod 2.
Operations for n = 4:
Move disk 1 from rod 0 to rod 1.
Move disk 2 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 2.
Move disk 3 from rod 0 to rod 1.
Move disk 1 from rod 2 to rod 0.
Move disk 2 from rod 2 to rod 1.
Move disk 1 from rod 0 to rod 1.
Move disk 4 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 2.
Move disk 2 from rod 1 to rod 0.
Move disk 1 from rod 2 to rod 0.
Move disk 3 from rod 1 to rod 2.
Move disk 1 from rod 0 to rod 1.
Move disk 2 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 2.


## Advanced C++ techniques

### Function arguments: Passing by value, pointer, and lvalue and rvalue references

Here is what these different argument types mean:
- An argument passed by *value* is in general a copy of the object passed. If the function modifies the argument, the modification is not visible to the caller.
- An argument passed by *reference* **might** [\*] allow the function modifications on the argument become visible to the caller ([\*]: **will** if the reference is to a non-constant object, **will not** if the reference has the constant qualifier). There are two types of references, *lvalues* and *rvalues*:
  - An *lvalue* reference is what you will encounter most of the time, and this is what most people will refer to when they talk about references in C++. It means the object in question already has an address in the program, and the caller is simply pointing to it.
  - An *rvalue* reference does not yet have an address in the program. Examples are literals, function calls that return non-reference types, and temporary objects that are accessible only to the compiler.
- A *pointer* is an object the value of which is an address in memory. Passing a pointer with a valid memory address acts in the same way as passing by *lvalue* reference.

Compile `exercise_fcnargs.cc` in the repository, identify the different types of arguments, and run it to see what comes out:

#### Possible printout:

```
Value of n: 3, address of n : 0059FB7C
Pass by value: 3, address: 0059FB58
Pass by value: 5, address: 0059FB58
Value of ptr_n: 0059FB7C, value of its object: 3
Passed a constant reference to a pointer 0059FB7C with value *n = 3
Value of ref_n: 9, address of ref_n: 0059FB7C
Passed pointer 0059FB7C to constant value *n = 9
Value of d: 3.14, address of d: 0059FB6C
Pass by non-const lvalue reference: 3.14, address: 0059FB6C
Value of dc: 2.3, address of dc: 0059FB64
Pass by const lvalue reference: 2.3, address: 0059FB64
Pass by non-const rvalue reference: 4.5, address: 0059FB5C
Pass by non-const rvalue reference: 7.2, address: 0059FB6C
Value of d (= 3.14) after call with std::move(d): 9.3, address: 0059FB6C
Pass by const rvalue reference: 2.3, address: 0059FB64
Value of dc (= 2.3) after call with std::move(dc): 2.3, address: 0059FB64
```

In this example,
- Note that passing the variables `d` and `dc` to `std::move` marked them as *transferable*. For that reason, the functions with *rvalue* references are called the second time.
- Notice also the difference between arguments `int const* n` (also could be written as `const int* n`) and `int* const& n`. In the first case, the pointer points to a constant integer, whereas in the second case, the pointer is constant, but its value is **not**. In general, read parameter types *from right to left* after putting qualifiers (`const`) to the right, e.g., read `const int*` $\to$ `int const*` as "pointer to constant integer", and `int* const&` as "reference to constant pointer to an integer".

  - Try reading `const int* const&`.

### Templates in C++

You may want to define functions and classes for multiple types. One way to do it is
```
void print_value(int const& n){/* Do something for integer arguments */}
void print_value(double const& n){/* Do something similar for double-precision floating type arguments */}
```
For a small number of choices, this is probably faster to code. What happens if you have a plethora of types? The general solution is to use "template"s.

Examine `exercise_templates.cc` and run it.

#### Expected printout:
```
Value is 5.
Value is 2.3.
Twice the value of the float 5.1 is 10.2.
Value is 7.25.
Value is 1.221.
Twice the value of the float 1.221 is 2.442.
Value is -87.
```

The syntax `template<typename T> ...` indicates that what you are about to declare or define will use an arbitrary typename T. If type T is a class with member functions, every call to its member functions needs to be valid. For example, if you call `x.value()` for `x` of type `T`, `T` must have this function, e.g., `T` cannot be a `float`.

Templates can be specialized. Take a look at the syntax of `template<> void print_value(float const& x)`. `template<>` indicates this is a specialization. In function specializations, the arguments should be of the same general signature. For a templated definition `template<typename T> void print_value(T const& x)`,
- this is a valid specialization: `template<> void print_value(float const& x)` (or `template<> void print_value<float>(float const& x)` to write it more explicitly).
- this is not: `template<> void print_value(float&& x)`.

If you want to have `void print_value(float&& x)`, you can simply define it without `template<>`. That would make it an **overload**, not a *specialization*.

Function template specializations have to be *complete*. You cannot have, for example, `template<typename T> void print_value<TestClass<T>>(TestClass<T> const& x)`. For each possible type T that TestClass can take, you must define `print_value<TestClass<T>>` separately.

That naturally brings us to the specialization of classes. Here, you can have **partial-specializations**. As you can see in the example, for the template class `template<typename T> struct TestClass`, we could define fully- and partially-specialized cases:
- Full specialization for `T` $\to$ `float`: `template<> struct TestClass<float>`.
- Partial specialization for `T` $\to$ `TestClass<T>`: `template<typename T> struct TestClass<TestClass<T>>`. Notice here that the template argument list is not empty and contains a different, arbitrary type `T`.

#### But I really want partially-specialized functions!

Sure you do, and there is a way based on what you have just learned.

**Exercise**: Find a way to print something different for types `TestClass<T>` and `TestClass<TestClass<T>>` for an arbitrary type `T`, and a default way for any other type, by passing the object to print by reference.

&emsp;**Bonus**: Try to do the task of printing without spending a single byte in memory.

#### One possible solution

```
template<typename T> struct print_value_s{
  static void op(T const& x){
    std::cout << "Generic value of x = " << x << std::endl;
  }
};
template<typename T> struct print_value_s<TestClass<T>>{
  static void op(TestClass<T> const& x){
    std::cout << "TestClass<T>.x = " << x << std::endl;
  }
};
template<typename T> struct print_value_s<TestClass<TestClass<T>>>{
  static void op(TestClass<TestClass<T>> const& x){
    std::cout << "TestClass<TestClass<T>>.x = " << x << std::endl;
  }
};
template<typename T> void print_value(T const& x){ print_value_s<T>::op(x); }
// Without the bonus: Remove the static qualifiers, and define last function as
// template<typename T> void print_value(T const& x){ print_value_s<T>().op(x); }
```

Notice in this solution that I have used `static` functions to avoid creating a class object explicitly. The struct `print_value_s` has no data members, so any object of this type occupies 1 byte in memory, which is not really needed if you can declare all of your functions statically.

### "Substitution Failure Is Not An Error" (SFINAE)

Templated constructs can be used in conjunction with what is known as "Substitution Failure Is Not An Error" (SFINAE) to allow different template overloads.

Let's go over the example from https://en.cppreference.com/w/cpp/language/sfinae:

```
#include <iostream>
 
// This overload is added to the set of overloads if C is
// a class or reference-to-class type and F is a pointer to member function of C
template<class C, class F>
auto test(C c, F f) -> decltype((void)(c.*f)(), void()){
  std::cout << "(1) Class/class reference overload called\n";
}
 
// This overload is added to the set of overloads if C is a
// pointer-to-class type and F is a pointer to member function of C
template<class C, class F>
auto test(C c, F f) -> decltype((void)((c->*f)()), void()){
  std::cout << "(2) Pointer overload called\n";
}
 
// This overload is always in the set of overloads: ellipsis
// parameter has the lowest ranking for overload resolution
void test(...){
  std::cout << "(3) Catch-all overload called\n";
}
 
int main(){
  struct X { void f() {} };
  X x;
  X& rx = x;
  test(x, &X::f);  // (1)
  test(rx, &X::f); // (1), creates a copy of x
  test(&x, &X::f); // (2)
  test(42, 1337);  // (3)
}
```
Things to note in this example:
1. The expression `(A, B)` in attempts to evaluate A, discards its result, and returns the result of evaluating B. This is the general use case of the *comma operator* in C++.
1. `decltype` receives a single argument, but because of the comma operator, it appears as if taking two arguments. `decltype(..., void())` evaluates to `void`.
1. Both template functions have the same signatures in their arguments. Normally, this would not be allowed. However, in the way their return types are specified, notice that `(c.*f)` and `(c->*f)` cannot evaluate correctly at the same time. If `C` is a class or class reference type, only `(c.*f)` can evaluate. If, on the other hand, `C` is of pointer type, `(c->*f)` would be the one that evaluates correctly. Because the remaining subsitution has failed and "substitution failure is not an error", the compiler discards the failed alternative, and the code compiles without a complaint!

Here is another interesting way to overload template specializations of functions:
```
#include <iostream>
#include <type_traits>

struct Base_A{
  int a;
  Base_A(int a) : a(a){}
}
struct Base_B{
  double b;
  Base_A(double b) : b(b){}
}

template< typename T, std::enable_if_t<std::is_arithmetic_v<T>, bool> = true > void print_value(T const& x){ std::cout << "Arithmetic x = " << x << std::endl; } // (1)
template< typename T, std::enable_if_t<std::is_base_of_v<Base_A, T>, bool> = true > void print_value(T const& x){ std::cout << "x.a = " << x.a << std::endl; } // (2)
template< typename T, std::enable_if_t<std::is_base_of_v<Base_B, T>, bool> = true > void print_value(T const& x){ std::cout << "x.b = " << x.b << std::endl; } // (3)

int main(){
  print_value(3); // Calls (1)
  print_value(Base_A(3)); // Calls (2)
  print_value(Base_B(3)); // Calls (3)
}
```
Here, `enable_if_t<...>` is a shorthand for `typename enable_if<...>::type`.

`enable_if` has the type `type` defined as the second argument passed if the first argument evaluates to `true`. Otherwise, there is no type definition `type`.

For example, `std::is_base_of_v<Base_A, int>` is `false`, so `std::enable_if_t<std::is_base_of_v<Base_A, T>, bool> = true` evaluates to `std::enable_if_t<false, bool> = true` $\to$ `(undefined substitution) = true`. Since SFINAE, the compiler ignores this option and searches for another possibility.

When evaluating `std::enable_if_t<std::is_arithmetic_v<T>, bool> = true`, `std::is_arithmetic_v<int>` is ` true`, so `std::enable_if_t<std::is_arithmetic_v<T>, bool> = true` $\to$ `bool = true`. There is no substitution failure in this option, so the compiler happily picks it.

SFINAE can also be used in determining whether a class defines a public type, and acquire that type with a default otherwise. Consider this example:

```
template<typename T> class get_value_type{
private:
  template<typename U> static auto test(int) -> typename U::value_type;
  template<typename U> static auto test(...) -> T;
public:
  using type = decltype(test<T>(0));
};
```

Here, SFINAE is used to determine whether `test<T>(0)` is meant to invoke `test(int)` or `test(...)`. If the type `T` has `T::value_type` defined, `get_value_type<T>::type` would be `T::value_type`; otherwise, it would be the same as `T`.

#### Exercise:

Find a way to learn whether an arbitrary class has a public member function `value()`.

#### One possible solution:

```
template<typename T> struct has_call_value{
  struct invalid_call_type{};
  template <typename U> static constexpr auto test(int) -> decltype(&U::value);
  template <typename U> static constexpr auto test(...) -> invalid_call_type;
  static constexpr bool value = !std_ttraits::is_same_v<invalid_call_type, decltype(test<T>(0))>;
};
template<typename T> inline constexpr bool has_call_value_v = has_call_value<T>::value;
```

### The `virtual` keyword

Try to compile the following program:

```
#include <iostream>

struct A{
  int x;
  A() : x(5){}
};
struct B : A{};
struct C : A{};
struct D : B, C{};

int main(){
  D d;
  std::cout << d.x << std::endl;

  return 0;
}
```

As you might observe, it will not compile, complaining that the request to member `x` is ambiguous. The reason is that there are two instances of struct `A` created in this inheritance pattern, one from `B` and another from `C`. This is a classic diamond problem in C++.

To make sure you have a single instance of `A`, add the keyword `virtual` when inheriting the common base classes. That would ensure 'virtual inheritance'.

In the above example, that would mean making the following change:

```
struct B : virtual A{};
struct C : virtual A{};
```

Try to compile again with this change.

You can also use the `virtual` keyword to mark a member function of a base class so that daughter classes can redefine it. If a virtual function is declared as unimplemented by setting to 0, the class becomes an `abstract` class, which is a way to ensure developers provide a definition for this routine. Here is a very simple demonstration:

```
struct A{
  int x;
  virtual int get() = 0; // This line makes class Base an `abstract` class.
  virtual void set(int x_){ x = x_; }
}
struct B : A{
  int get(){ return x; }
}
struct C : A{
  int get(){ return 5; } // Returns 5 regardless of the value of A::x.
  void set(int){ x = 5; } // Will also reset A::x to 5.
}
```




Since pointers to addresses of objects of derived classes can point to the base class, deleting these objects using the base class pointers may lead to undefined behavior. Consider this example:

```
#include <iostream>

struct A{
  ~A(){ std::cout << "Called A::~A()" << std::endl; }
}
struct B{
  int* x;
  B(int x_) : x(new int(x_)){}
  ~B(){ std::cout << "Called B::~B()" << std::endl; delete x; }
}

int main(){
  A* a = new B(5);
  delete a; // Memory leak!
  return 0;
}
```

Here, there is a memory leak when deleting the pointer to `A` because the base class `A` has no idea that the derived class `B` is also being destructed, which would have deleted the member `x` properly.

To call the destructor of the derived class properly, and avoid the memory leak in this example, you could declare the destructor of `A` to be virtual:

```
struct A{
  virtual ~A(){ std::cout << "Called A::~A()" << std::endl; }
}
```

That would properly call the destructor `B::~B()`. You can compile with the modification and see how the printout changes.

### Curiously Recurring Template Patterns (CRTPs)

Virtual functions provide 'runtime` polymorphisms, which require virtual tables to resolve function calls to each polymorphic object.

An alternative is to use a 'Curiously Recurring Template Pattern' (CRTP) to provide an implementation. This is a useful method if you do not want the overhead that results from virtual function lookups and `dynamic_cast`s, or if dynamic polymorphism is undesirable (we will see one important case during the next session).

Here is an example:

```
template<typename Derived> class A : public Derived{
public:
  void print(){ static_cast<Derived*>(this)->print_impl(); }
}
class B : public A<B>{
public:
  void print_impl(){ std::cout << "Hello there!" << std::endl; }
}
```

By letting the base class derive from a daughter class, one could inherit the implementation function, which would allow the call to `print()` to be differentiated for different daughter classes.